<p><img src="imagenes/cabecera.png" width="900" align="center"></p>

# Trabajo práctico 1: Análisis de sentimientos

## Curso Procesamiento de Lenguaje Natural 

### Maestría en Tecnologías de la información



**Trabajo práctico porpuesto por:** Julio Waissman Vilanova (julio.waissman@unison.mx)

**Desarrollado por:** _Poner tu nombre y correo electrónico aquí_


En este trabajo práctico tiene como objetivo el analizar y entender la importancia de cada paso que se debe llevar para la clasificación de textos. Los pasos que queremos analizar son:

1. Normalización de datos
3. *Stemming*
4. Tokenización
4. Extracción de características
5. Clasificación

Por esta razón, no vamos a dedicar tiempo a otras dos taeras fundamentales y que pueden consumir más de la mitad del tiempo de un proyecto de PLN: La obtención y limpieza de los datos. Por esta razón, vamos a mantenernos con el mismo conjunto de aprendizaje de las libretas del curso (TASS2015, tarea 1). 

La importancia de la libreta es experimentar cuales son las modificaciones que más impactan al resultado y cuales cambios no presentan modificaciones realmente. 

## 1. Cargando los datos y separando el conjunto de prueba y el de aprendizaje

In [ ]:
# Esta etapa ya la realizamos en forma estandar, vamos a cargar aquí todas los módulos a utilizar
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import sklearn
from ast import literal_eval 
plt.rcParams['figure.figsize'] = (20, 12)

# Carga el corpus del archivo pickle
df_train = pd.read_pickle('datos/tass2015/general-tweets-train-dt.pkl')
df_test = pd.read_pickle('datos/tass2015/general-tweets-test-dt.pkl')

# Extrae los datos de entrenamiento y prueba para análisis de sentimiento
x = dt_train['texto'].values
x_test = dt_train['texto'].values
y_polaridad = df_train['polaridad'].values

# Separa los datos en conjunto de entrenamiento y validación
x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(
    x, y_polaridad, test_size=0.1, random_state=10
)

## 2. Función de normalización de datos

**Aquí pongo una función de normalización muy sencilla, pero la idea es que esta se modifique en función de las necesidades**

Algunas ideas:

1. ¿Hay ventajas en poner el texto en minúsculas o no?
2. ¿Es mejor eliminar los usuarios y los *hashtags*, es mejor ponerlos con un nombre generico (USR o URL), o es mejor dejarlos como están?
3. ¿Es mejor eliminar todos los signos de puntuación? ¿Mejor recuperar alguno?
4. ¿Que pasa si los emoticones los cambiamos por las palabras `feliz` o `triste`?
5. ¿Vale la pena eliminar las *palabras vacias*

Escribe a continuación al menos una idea de normalización que consideres puede ser útil. Es muy importante que esto lo hagas *antes* de programarlo y probarlo. No importa si son o no útiles al final, lo importante en este caso es adquirir experiencia práctica.

1. _Escribe una propuesta a modificar_
2. ...

Por último, vamos a agregar la opción de realizar el proceso de *steeming* o no, utilizando el método clásico propuesto para español por *Snowball*.

In [ ]:
# Las expresiones regulares precompiladas es mejor realizarlas fuera de la función por eficiencia
remplaza_por_espacios_re = re.compile('[\n/(){}\[\]\|@,;\.]')
simbolos_a_eliminar_re = re.compile('[^\d\w #+_]')
palabras_vacias = set(nltk.corpus.stopwords.words('spanish'))

#TODO: Modifica la normalización de texto
def normaliza_texto(texto, stemmer=None):
    """
    Normaiza un documento
    
    :param texto: Una cadena de caracteres
    :param steemer: Una objeto steemer, si es None, entonces no hace steeming del texto
    
    :return: Una cadena de caracteres con el texto modificado
    """
    text = texto.lower()
    
    # AGREGA AQUI EL CODIG QUE SEA NECESARIO Y EN EL ORDEN NECESARIO

    # Eliminación de simbolos
    text = re.sub(remplaza_por_espacios_re, ' ', text)
    text = re.sub(simbolos_a_eliminar_re, '', text)
    
    # Elimina palabras de paro
    text = ' '.join([palabra for palabra in text.split() 
                     if palabra not in palabras_vacias])
    
    # Steeming
    if stemmer is not None:
        text = ' '.join([stemmer.stem(palabra) for palabra in text.split()])
    
    return text

Es muy importante, en cada paso, hacer una prueba para ver si el método está funcionando como lo diseñaste. En particular las expresiones regulares dan muchas sorpresas. Vamos a seleccionar un documento al azar y lo vamos a normalizar. Realiza esto varias veces para ver diferentes documentos y estar seguro el lo que programaste funcione correctamente.

En su lugar puedes poner un ejemplo específico con los detalles que quieras revisar.

In [ ]:
doc = np.random.choice(x_train)
doc_norm = normaliza_texto(doc)

print("{}\n\n{}".format(doc, doc_norm))

Y ahora con *stemming*

In [ ]:
stemmer = nltk.stem.snowball.SpanishStemmer()
doc = np.random.choice(x_train)
doc_norm = normaliza_texto(doc, stemmer)

print("{}\n\n{}".format(doc, doc_norm))

Para poder aplicar este método a todo el ndarray de nuestro corpora, hay que vectorizar la funcion (y así llamarla para todo elemento). 

In [ ]:
normaliza_texto_np = np.vectorize(normaliza_texto, excluded=['stemmer'])

# Ejemplo
normaliza_texto_np(x_train[:5], stemmer)

## 3. Tokenización y extracción de características

**Esta funcion la debes de completar, algunas cosas se codifican internamente**

Para la tokenización y extracción de características vamos a utilizar los métodos que provee *sklearn*, los cuales los vamos a ajustar de acuerdo a parámetros que escojamos. 

Entre las desiciones que hay que tomar es:
1. ¿Cual metodo de tokenización? La tokenización en los métodos incluidos en *sklearn* se definen por expresiones regulares. Por default, *sklearn* considera un token solo aquellos grupos de al menos dos letras, pero elimina todos los símbolos y los números. Esto en algunos casos puede no ser la mejor opción.

2. Tipo de tokenizador. Vamos a mantener tres posibles tokenizadores:
    a. BOW
    b. Cuentas (BOW con frecuencia del token en el documento)
    c. TF-IDF
    
3. Número mínimo de documentos en los que debe aparecer un token para ser considerado, y porcentaje máximo de documentos en los que un token aparezca para considerarlo con poder de discriminación.

4. Número mínimo y máximo de $n$-gramas a considerar.


In [ ]:
def extracion_caracteristicas(x, tipo="BOW", token_re=r'(?u)\b\w\w+\b', min_df=5, max_df=0.9, bigramas=False):
    """
    Genera un objeto tipo CountVectorizer o TdfdfVectorizer
    
    :param x: Un ndarray con el texto de entrenamiento
    :param tipo: Una cadea que puede ser "BOW", "Count" o "TF-IDF"
    :param token_re: Una expresión regular que defina los tokens (por default r'(?u)\b\w\w+\b')
    :param min_df: int, número mínimo de documentos donde debe aparecer el token (5 por default)
    :param max_df: float 0.0 <= max_df <= 1.0, porcentaje máximo donde aparece un token significativo (0.9 default)
    :param bigramas: bool, True si se incluyen unigramas y bigramas, False, solo unigramas
    
    :return un objeto tipo CountVectorizer
    
    """
    ngram_range = (1,2) if bigramas else (1,1)
    
    if tipo is "BOW":
        vectorizer = sklearn.feature_extraction.text.CountVectorizer(
            analyzer='word', binary=True, token_pattern=token_re, 
            min_df=min_df, max_df=max_df, ngram_range=ngram_range            
        )
    elif tipo is "Count":
        vectorizer = None # <----- Desarrollar aqui
    elif tipo is "TF-IDF":
        vectorizer = None # <----- Desarrollar aqui
    else:
        raise NotImplementedError("El método {} no está contemplado".format(tipo))
    vectorizer.fit(x)
    
    return vectorizer

Para probarlo vamos a extraer características de los primeros 50 ejemplos. 

In [ ]:
t_re = None # <-- Cambialo aqui para probar con otros me´todos de tokenización

x_ejemplo = normaliza_texto_np(x_train[:50])
if t_re is None:
    vectorizer = extracion_caracteristicas(x_ejemplo, min_df=1, max_df=1.0)
else:
    vectorizer = extracion_caracteristicas(x_ejemplo, min_df=1, max_df=1.0, token_re=t_re)
    

Ahora **contesta las siguiente preguntas** (Aquí mismo)

1. ¿Cuantas palabras tiene el vocabulario con el método de tokenización por default?
2. ¿Cuantas palabras tiene el vocabulario si onsideras como token cualquie serie de uno o más
   signos diferentes a un simbolo de espacio en blanco?
3. Si entrenas con el método de 'TF-IDF' ¿Cuales son los tres tokens con mayor valor?
4. Responde a la misma pregunta, pero en el caso que incluyas bigramas
5. ¿Cual es el token que más aparece en cada caso?

Agrega aqui abajo el código necesario para responder a las preguntas

In [ ]:
# Agrega aqui todo el código necesario




## 4. Clasificación

En la libreta del curso practicamos utilizando un clasificador logístico, el cual no ajustamos, sin embargo existen otros clasificadores, los cuales pueden tener un resultado. En particular nos interesa el método de máquinas de vectores de soporte (SVM por Support Vector Machines), las cuales se utilizan también en forma usual para la clasificación de texto. Para una explicación sobre como funcionan las máquina de vectores de soporte se puede consultar este [capítulo](https://see.stanford.edu/materials/aimlcs229/cs229-notes3.pdf) que a mi me gusta mucho.

Lo mejor de utilizar los métodos existentes en *sklearn* es que solamente se requiere entender la idea general del método y un poco de intuición sobre como funcionan los parámetros. En la [documentación de *sklearn*](http://scikit-learn.org/stable/modules/svm.html#svm) se encuentra una explicacion de las SVM para la clasificación con múltiples clases. De la documentación podemos concluir que existen dos algoritmos:

1. Un algoritmo para la aproximación lineal (similar a la regrasión logística) con `LinearSVC`. Para este algoritmo hay dos parámetros a ajustar: la intensidad de la regulación `C` y la norma para la penalización `penalty`, la cual puede ser `l2` o `l1`(exactamente como en la regresión logística).

2. Un algoritmo para la aproximación utilizando un kernel (el más utilizado es el gaussiano, por default). En este caso el único parámetro que vamos a ajustar es `C`.


Ahora es necesario desarrollar una función que nos devuelva el tipo de clasificador que queremos, ya entrenado. **Desarrolla la función aquí abajo**

In [ ]:
def genera_clasificador(x, y, tipo='logistica', C=1.0, penalty='l2'):
    """
    Genera un clasificador segun el tipo que escojamos, y lo preentrenamos con x, y y
    
    :param x: ndarray de shape (n_ejemplos, n_caracteristicas) con los ejemplos de entrenamiento
    :param y: ndarray de shape (n_ejemplos,) con las clases asignadas al conjunto de entrenamiento
    :param tipo: tipo in ['logistica', 'SVClin, 'SVCgauss'] con el tipo de clasificador a utilizar
    :param C: float, con la fuerza de la regularización
    :param penalty: penalty in ['l2', 'l1'] la norma de la penalización, no aplica para SVC
    
    :return un objeto de una clase heredada de ClassifierMixin, con un clasificador de sklearn y sus 
            métodos estandar
    
    """
    if tipo is 'logistica':
        clf = sklearn.linear_model.LogisticRegression(C=C, penalty=penalty)
    elif tipo is 'SVClin':
        clf = None # <---- COMPLETAR EL CÓDIGO
    elif tipo is 'SVCgauss':
        clf = None # <---- COMPLETAR EL CÓDIGO
    else:
        raise NotImplementedError("El clasificador {} no lo tenemos disponible todavía".format(tipo))
        
    clf.fit(x, y)
    return clf

Y por último copiamos el código de reporte de resultados, aunque con una modificación.

In [ ]:
def reporte(y_real, y_estimada, labels=None):
    print("\nPorcentaje de acierto: {}".format(sklearn.metrics.accuracy_score(y_real, y_estimada)))
    print("\nPrecisión, recall y f1-score")
    print(sklearn.metrics.classification_report(y_real, y_estimada, target_names=labels))    


## 5. Poniendo todo junto

Ahora vamos a analizar cuales son los factores que más afectan en el análisis de sentimientos. Para eso vamos a poner todo junto.

In [ ]:
## PARAMETROS SELECCIONADOS 

# Normalización
stemmer = None 

# Tokenización
token_re = r'(?u)\b\w\w+\b'

# Extracción de características
tipo_vec = "BOW"
min_df = 5
max_df = 0.9
bigramas = False

# Clasificación
tipo_clf = 'logistica'
C = 1.0
penalty = 'l2'

## EL PROCESO
x_train_norm = normaliza_texto_np(x_train, stemmer) 
x_val_norm = normaliza_texto_np(x_val, stemmer)

vectorizador = extracion_caracteristicas(
    x_train_norm, tipo=tipo_vec, token_re=token_re, 
    min_df=min_df, max_df=max_df, bigramas=bigramas
)
x_train_vec = vectorizador.transform(x_train_norm)
x_val_vec = vectorizador.transform(x_val_norm)

clasificador = genera_clasificador(
    x_train_vec, y_train, tipo=tipo_clf, C=C, penalty=penalty
)
y_est_train = clasificador.predict(x_train_vec)
y_est_val = clasificador.predict(x_val_vec)

print("Resultados con los datos de entrenamiento")
reporte(y_train, y_est_train)
print("Resultados con los datos de entrenamiento")
reporte(y_val, y_est_val)

### Contesta las sigientes preguntas

1. Manteniendo los parámetros de extracción de características y los del clasificador por default, ¿Cuales son los métodos de normalización, combinados con el tokenizador, que mejores resultados dan? Explica al menos unos 5 cambios que hayas probado y que influyeron mucho en el resultado final, ya sea para bien, ya sea para mal. Recuerda que las operaciones sobre la normalización las tienes que realizar directamente en la función correspondiente.
    1. _Primer modificacion explicada_
    2. _Segunda modificación explicada_
    3. _Tercera modificación explicada_
    4. _Cuarta modificación explicada_
    5. _Quinta modificación explicada_
    6. _Agrega cuantas consideres interesantes_

2. ¿Existe una diferencia notable entre utilizar BOW, Count y TF-IDF? ¿Cual es el mejor método en este problema particular?

3. ¿Cual es la mejor selección de min_df, max_df? ¿Tienen una influencia importante sobre los resultados?

4. Completa la siguiente tabla

| Vectorizador     | Clasificador  | Mejor C | Mejor penalty | *Accuracy* | F1-score |
| ---------------- |:------------- | :------ | :-----------: | :--------- | :------- |
| TF-IDF unigramas | Logístico     |         |               |            |          |
| TF-IDF bigramas  | Logístico     |         |               |            |          |
| TF-IDF unigramas | SVC lineal    |         |               |            |          |
| TF-IDF bigramas  | SVC lineal    |         |               |            |          |
| TF-IDF unigramas | SVC gauss     |         |   No aplica   |            |          |
| TF-IDF bigramas  | SVC gauss     |         |   No aplica   |            |          |

Si con tu normalización, el mejor método de extracción de características no es $TF-IDF$ tienes toda la libertad de cambiar el vectorizador de la tabla


## Por último escribe aqui un párrafo breve con tus conclusiones


--Ingresar el texto aquí--